# 공간 데이터의 연산에 대하여

1. 왜 공간 데이터의 연산을 알아야 하는가? 
- 라벨의 면적과 길이를 구해야 한다.
- 인접하는 영상에서 경계선에 있는 라벨들을 처리해야 한다. ex) 경계선에서 잘려버린 빌딩들, n개의 블록으로 나뉘어진 하나의 도로...
2. 그러면 공간 데이터의 연산이 무엇인가? 
- 공간 데이터의 기하적 요소를 가지고 계산을 하는 것이다. 
- 2차원 이상의 공간에서 정의할 수 있다. (0차원 점, 1차원 선, 2차원 면, 3차원 부피를 가진 객체)
    - 면의 넓이
    - 면을 이루는 변의 길이
    - 교차하는 선의 지점 
- 다른 기하 객체와의 관계성을 정의하는 연산들이 있다.(위상학, topology)
    - 인접(touch)하는가? 
    - 겹치는가?(intersect) 
    - A가 B를 포함(overlap)하는가?
    - A와 B간의 거리는 얼마나 되는가?
3. 필요성과 예시는 알겠으나 우리가 필요한 연산은 무엇인가?
- 길이와 넓이를 구하는 연산과 인접하는지 여부, 기하끼리 겹치는지의 여부를 확인
    - Overlap : 기하끼리 겹치는가? 
    - Touch : 경계선(boundary)끼리 한 점에서 만나는가? 
- 한 빌딩이지만 영상의 경계선에 존재하여 라벨은 2개로 쪼개어져 버린 빌딩을 하나로 인식하는 데 필요하다. 
    - spacenet 데이터를 사용하면 영상이 이어져 있는 경우가 많아서 이를 고려하는 것이 좋다(후순위)
4. 어떻게 공간 연산을 시행할 수 있는가?
- PostGIS(Postgresql이라는 오픈소스 데이터베이스의 공간 데이터 확장)에서 제공하는 공간 연산 쿼리를 이용한다. 
    - 장점 : 빠르다. 최적화된 연산 제공.  
    - 단점 : 배경지식이 다소 필요하다. DB에 데이터를 어떻게 넣어야 연산에 유리한지 고려해야 한다.(단점은 아니나 생각해볼 만하다.)
    - 사용 예시 : https://programmerpsy.tistory.com/97?category=1010453
- **GeoPandas : Pandas + 공간 정보 저장 특화 기능들**
    - 장점 : 기존 Pandas를 계승한 함수들이 많아서 판다스에 익숙하면 쓰기 쉽다. 
    - 단점 : `GeoPandas`라고 붙어있는 만큼, 반드시 dataframe에 기하 정보가 들어가 있어야 GeoPandas를 쓸 수 있다. 그리고 오픈소스인 만큼 오류가 난무한다...정상적으로 동작하지 않으면 기존 Pandas 함수를 사용해서 해결할 수 없는지 알아보자. 
    - 사용 예시 : https://programmerpsy.tistory.com/105?category=1010453
- GDAL : 공간 정보에서 가장 유명한 라이브러리 
    - 장점 : 다양한 연산들과 데이터 포멧을 지원해서 편하다. 
    - 단점 : 설치 난이도 중상, 사용 난이도 상.
    - 사용 예시 : 

## 두 polygon이 겹치는 경우
### 필요한 라이브러리 설치 스크립트
### 필요한 라이브러리 설치 스크립트
- gdal
```
sudo add-apt-repository ppa:ubuntugis/ppa && sudo apt-get update
sudo apt-get update
sudo apt-get install gdal-bin
sudo apt-get install libgdal-dev
export CPLUS_INCLUDE_PATH=/usr/include/gdal
export C_INCLUDE_PATH=/usr/include/gdal
```
여기까지 하고 gdal 버전을 확인한 후 아래의 명령어를 시행한다. 
```
gdal-config --version
```
gdal 버전 확인하고
```
pip install GDAL=={version}
```
예를 들면 `pip install GDAL==2.3.3`이렇게 해주면 이전에 설치해주었던 GDAL 라이브러리와 python용 GDAL 라이브러리 버전이 맞아서 제대로 작동 가능하다. 
```

In [5]:
# 예시 코드
from osgeo import ogr

wkt1 = "POLYGON ((1208064.271243039 624154.6783778917, 1208064.271243039 601260.9785661874, 1231345.9998651114 601260.9785661874, 1231345.9998651114 624154.6783778917, 1208064.271243039 624154.6783778917))"
wkt2 = "POLYGON ((1199915.6662253144 633079.3410163528, 1199915.6662253144 614453.958118695, 1219317.1067437078 614453.958118695, 1219317.1067437078 633079.3410163528, 1199915.6662253144 633079.3410163528)))"

poly1 = ogr.CreateGeometryFromWkt(wkt1)
poly2 = ogr.CreateGeometryFromWkt(wkt2)

# intersect가 되지 않으면 None을 반환한다.
intersection = poly1.Intersection(poly2)
 
print(intersection.ExportToWkt())

POLYGON ((1208064.27124304 614453.958118695,1208064.27124304 624154.678377892,1219317.10674371 624154.678377892,1219317.10674371 614453.958118695,1208064.27124304 614453.958118695))
